In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
#from qat.core.console import display

In [ ]:
%load_ext qat.core.magic

#QPU connection
try:
    from qat.qlmaas import QLMaaSConnection
    connection = QLMaaSConnection()
    LinAlg = connection.get_qpu("qat.qpus:LinAlg")
    lineal_qpu = LinAlg()
except ImportError:
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()  

In [ ]:
#Libreria Zalo
sys.path.append('/home/cesga/gferro/NEASQC/PhaseAmplification/')
#Libreria Juan
sys.path.append('/home/cesga/gferro/NEASQC/ProgramasDefinitivos/')

# Quantum Phase Amplification

Voy a comparar los operadores que monta Juan para la *Phase Amplification* con respecto a los que implementé yo.

El notebook *QPA04_PuertasBase.ipynb* tiene toda la teoría y la implementación de las diferentes puertas por lo que aquí voy a aligerar todo lo que pueda para hacer testeos rápidos.

## 1. Carga Completa

Lo primero que necesitamos es realizar una carga completa de los datos sobre el sitema cuántico. Esta operación se puede resumir del siguiente modo:

$$|\Psi\rangle_{n+1}=\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)|0\rangle_{n}\otimes|0\rangle_{1}$$


### 1.1 Preparo Datos

In [ ]:
from AuxiliarFunctions import  get_histogram, PostProcessResults, TestBins, RunJob
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 4
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

### 1.2 Circuto de Carga de datos

In [ ]:
from QuantumMultiplexors_Module_examples import LoadingData

In [ ]:
qprog = LoadingData(p_X, f_X)
circuit = qprog.to_circ()
%qatdisplay circuit 
job = circuit.to_job()
InitialState = PostProcessResults(RunJob(lineal_qpu.submit(job)))

In [ ]:
InitialState.head()

## 2. Amplificación de Amplitud

Después de la carga de datos tenemos el sistema en el estado $|\Psi\rangle_{n+1}$ que se puede descomponer en dos estados ortogonales $|\Psi_{1}\rangle$ y $|\Psi_{0}\rangle$ del siguiente modo

$$|\Psi\rangle_{n+1}=\sqrt{a}|\Psi_{1}\rangle+\sqrt{1-a}|\Psi_{0}\rangle$$

Donde 

$$|\Psi_{1}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\frac{1}{\sqrt{a}}\sqrt{p(x)f(x)}|1\rangle$$
$$|\Psi_{0}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\frac{1}{\sqrt{1-a}}\sqrt{p(x)(1-f(x))}|0\rangle$$



Donde $a=\sum_{x=0}^{2^{n}-1}p(x)f(x)$

Es decir los $n$ primeros qbits están en una superposición de estados y lo único que los diferencia es el estado el último qbit!!

Como la integral es la amplitud del estado $|\Psi_{1}\rangle$ el paso siguiente es intentar maximizar la probabilidad de que al medir obtengamos dicho estado aplicando el algoritmo de Groover. 

El algoritmo de Groover adapatado a amplificación de fase consiste en aplicar un número óptimo de veces $k$ el operador $\hat{Q}$. Este operador se define como:

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

Los operadores $\hat{U}_{|\Psi_{2}\rangle}$ y $\hat{U}_{|\Psi_{0}\rangle}$ se construyen del siguiente modo:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$
$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle \langle \Psi|$$


### 2.1 Operador $\hat{U}_{|\Psi_{0}\rangle}$

Este operador se construye del siguiente modo:
$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$

Una operación que sería circuitable del siguiente modo:

$$\hat{U}_{|\Psi_{0}\rangle }=(\hat{I_{n}}\otimes X)(\hat{I_{n}}\otimes Z)(\hat{I_{n}}\otimes X)$$

La aplicación de este operador sobre $|\Psi\rangle_{n+1}$ es:

$$\hat{U}_{|\Psi_{0}\rangle} |\Psi\rangle_{n+1} = \sqrt{a}|\Psi_{1}\rangle-\sqrt{1-a}|\Psi_{0}\rangle$$

Es decir el operador $\hat{U}_{|\Psi_{0}\rangle }$ realizaría una reflexión en torno el eje definido por el estado $|\Psi_{1}\rangle$


In [ ]:
from PhaseAmplification_Module import U_Phi_0 
from expectation_module import load_U0

In [ ]:
qZalo = LoadingData(p_X, f_X)
qZalo.apply(U_Phi_0(n_qbits+1), qZalo.registers)
circuitZ = qZalo.to_circ()
%qatdisplay circuitZ 
job = circuitZ.to_job()
ZaloState = PostProcessResults(RunJob(lineal_qpu.submit(job)))

In [ ]:
qJuan = LoadingData(p_X, f_X)
U0_gate = load_U0(n_qbits)
qJuan.apply(U0_gate, qJuan.registers)
circuitJ = qJuan.to_circ()
%qatdisplay circuitJ 
job = circuitJ.to_job()
JuanState = PostProcessResults(RunJob(lineal_qpu.submit(job)))

### Comparaciones 

In [ ]:
InitialState.head()

In [ ]:
ZaloState.head()

In [ ]:
JuanState.head()

Basicamente el operador de Juan implementa la operación inversa al mío (Zalo). Mientras en el caso de zalo se implementa una reflexión en torno al estado $|\Psi_{1}\rangle$ en el caso de Juan la reflexión es en torno al estado $|\Psi_{0}\rangle$.
Zalo cambia de signos todos los estados con qbit final $|0\rangle$ mientras Juan cambia de signo los estados con qbit final $|1\rangle$

### 2.2 Operador $\hat{U}_{|\Psi\rangle}$

El operador $\hat{U}_{|\Psi\rangle}$ se basa en el operador difusor de Groover. Y su forma es la siguiente:

$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle \langle \Psi|$$

Como

$$|\Psi\rangle_{n+1}=\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)|0\rangle_{n}\otimes|0\rangle_{1}$$

Entonces podremos componer el operador del siguiente modo:

$$\hat{U}_{|\Psi\rangle } =\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)\hat{D}_{0} \left(\hat{P_{n}}\otimes I\right)^{\dagger} \hat{R_{n+1}}^{\dagger}$$

Donde $\hat{D}_{0}$ es una reflexion entorno al estador **perpendicular** al estado $|0\rangle_{n}$

$$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|$$

#### 2.2.1 Implementación  $\hat{D}_{0}$

Se puede demostrar que la implementación Circuital del Operador $\hat{D}_{0}$ es:

$$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|= \hat{X}^{\otimes n} c^{n-1}Z \hat{X}^{\otimes n}$$

In [ ]:
from PhaseAmplification_Module import LoadD0_Gate

In [ ]:
D0 = LoadD0_Gate(n_qbits)

In [ ]:
%qatdisplay D0 --depth 1

In [ ]:
qD0 = LoadingData(p_X, f_X)
qD0.apply(LoadD0_Gate(n_qbits+1), qD0.registers)

In [ ]:
circuitD0 = qD0.to_circ()
%qatdisplay circuitD0 

In [ ]:
job = circuitD0.to_job()
D0State =PostProcessResults(RunJob(lineal_qpu.submit(job)))

In [ ]:
D0State.head()

In [ ]:
InitialState.head()

In [ ]:
(D0State['Amplitude'].loc[1:] == InitialState['Amplitude'].loc[1:]).all()

In [ ]:
(D0State['Amplitude'].loc[0] == -InitialState['Amplitude'].loc[0])

Como comprobamos la puerta $D_0$ cumple su cometido!

#### 2.2.2 Implementación Circuital $\hat{U}_{|\Psi}\rangle$

Nos queda implementar el Difusor:
$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle \langle \Psi|$$

Como ya tenemos implementado

$$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|$$

y sabemos que:

$$|\Psi\rangle_{n+1}=\hat{R_{n+1}}\hat{P_{n}}|0\rangle_{n+1}$$

La forma rápida de implementar el operador Difusor: $\hat{U}_{|\Psi\rangle }$

$$\hat{U}_{|\Psi\rangle} = \hat{R_{n+1}}\hat{P_{n}}\hat{D}_{0} \hat{P_{n}}^{\dagger} \hat{R_{n+1}}^{\dagger}$$

Este operador se puede interpretar como una reflexión en torno al estado perpendicular a $|\Psi\rangle$


In [ ]:
from QuantumMultiplexors_Module import LoadP_Gate, LoadR_Gate

P_gate = LoadP_Gate(p_X)
R_gate = LoadR_Gate(f_X)

In [ ]:
from PhaseAmplification_Module import LoadD0_Gate
def U_Phi_Gate(P_gate, R_gate):
    """
    Create gate U_Phi mandatory for Phase Amplification Algorithm.
    The operator to implement is: I-2|Phi_{n-1}><Phi_{n-1}|. 
    Where the state |Phi_{n-1}> is: |Phi_{n-1}>=R*P*|0_{n+1}>. 
    Where R and P are the gates to load the integral of a function f(x) and 
    the load of a distribution probabilitiy p(x) respectively.
    Inputs:
        * nqbits: int. Number of Qbits of the Gate
        * P_gate: quantum gate for loading probability distribution.
        * R_gate: quantum gate for loading integral of a function f(x)
    Outputs:
        * U_Phi: guantum gate that implements U_Phi gate
    """
    
    
    from qat.lang.AQASM import AbstractGate, QRoutine
    nqbits = R_gate.arity
    def U_Phi_generator():
        """
        Circuit generator for the U_Phi_Gate.
        Operation to be implemented: R*P*D_0*P^{+}R^{+}
        Inputs:
            * nqbits: int. Number of Qbits for the circuit
        Outputs:
            * qrout: quantum routine with the circuit implementation
        """
        
        qrout = QRoutine()
        qbits = qrout.new_wires(nqbits)
        qrout.apply(R_gate.dag(), qbits)
        qrout.apply(P_gate.dag(), qbits[:-1])
        D_0 = LoadD0_Gate(nqbits)
        qrout.apply(D_0, qbits)
        qrout.apply(P_gate, qbits[:-1])
        qrout.apply(R_gate, qbits)
        return qrout
    U_Phi = AbstractGate(
        "UPhi", 
        [],
        circuit_generator = U_Phi_generator,
        arity = nqbits
    )
    return U_Phi()


In [ ]:
qPhi = LoadingData(p_X, f_X)
UPhi_Gate = U_Phi_Gate(P_gate, R_gate)
qPhi.apply(UPhi_Gate, qPhi.registers)
circuitPhi = qPhi.to_circ()
%qatdisplay circuitPhi --depth 0
job = circuitPhi.to_job()
PhiState =PostProcessResults(RunJob(lineal_qpu.submit(job)))

Cabe destacar que:
$$\hat{U}_{|\Psi\rangle } |\Psi\rangle = (\hat{I} - 2|\Psi\rangle \langle\Psi|)|\Psi\rangle=-|\Psi\rangle$$

Esto lo podemos utilizar para verificar que hemos implementado bien la puerta

In [ ]:
np.isclose(PhiState['Amplitude'],-InitialState['Amplitude']).all()

#### Testeo Implementación Juan

In [ ]:
from qat.lang.AQASM import Program
from expectation_module import load_probabilities, load_function, load_U0, load_U1, load_Q

centers, probs, P_gate = load_probabilities(n_qbits, p, LowerLimit, UpperLimit)
R_gate, y = load_function(centers, f, n_qbits)
Q_gate, U0_gate, U1_gate  = load_Q(n_qbits, P_gate, R_gate)

qU_J = Program()
qbits = qU_J.qalloc(n_qbits+1)
qU_J.apply(P_gate, qbits[:-1])
qU_J.apply(R_gate, qbits)
qU_J.apply(U1_gate, qbits)

In [ ]:
circuitqU_J = qU_J.to_circ()
%qatdisplay circuitqU_J 

In [ ]:
job = circuitqU_J.to_job()
result = lineal_qpu.submit(job)
qU_J_State = PostProcessResults(result.join())

In [ ]:
np.isclose(qU_J_State['Amplitude'],-InitialState['Amplitude']).all()